## Import Library

In [1]:
from machine_lib import * 

## 1, Login
在machine_lib文件的login方法中填写用户名和密码后保存然后来到本文件Restart Kernal后重新import machine_lib

In [2]:
s = login()

b'{"user":{"id":"TJ91365"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'


## 2, get data fields

In [3]:
def get_datasets(
    s,
    instrument_type: str = 'EQUITY',
    region: str = 'USA',
    delay: int = 1,
    universe: str = 'TOP3000'
):
    url = "https://api.worldquantbrain.com/data-sets?" +\
        f"instrumentType={instrument_type}&region={region}&delay={str(delay)}&universe={universe}"
    result = s.get(url)
    datasets_df = pd.DataFrame(result.json()['results'])
    return datasets_df

In [4]:
datasets_df = get_datasets(s) # by default we load all datasets USA TOP3000 delay 1
datasets_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

,id,name,description,category,subcategory,region,delay,universe,coverage,valueScore,userCount,alphaCount,fieldCount,pyramidMultiplier,themes,researchPapers
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",USA,1,TOP3000,0.6818,3.0,321,3887,197,1.2,[],"[{'type': 'discussion', 'title': 'Getting star..."
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,0.6203,2.0,677,43791,868,1.2,[],"[{'type': 'discussion', 'title': 'Getting star..."
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,0.9892,2.0,858,75662,2538,1.2,[],"[{'type': 'discussion', 'title': 'Getting star..."
3,analyst16,Real Time Estimates,This dataset provides real-time access to the ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",USA,1,TOP3000,0.8252,3.0,217,3288,107,1.2,[],"[{'type': 'research', 'title': 'Creating D0 Al..."
4,analyst35,ESG Model,The dataset provide ESG related information ba...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",USA,1,TOP3000,0.3771,4.0,69,455,46,1.2,[],"[{'type': 'discussion', 'title': 'Getting star..."


In [15]:
datasets = datasets_df[datasets_df['alphaCount']>10000]['id'].tolist()
len(datasets)

25

In [6]:
datasets[:5]

['analyst14', 'analyst15', 'analyst4', 'analyst7', 'fundamental13']

In [11]:
pc_fields = []
for dd in datasets:
    df = get_datafields(s, dataset_id = dd, region='USA', universe='TOP3000', delay=1)
    temp = process_datafields(df, "matrix") + process_datafields(df, "vector")
    pc_fields = pc_fields + temp

print(pc_fields[0])
len(pc_fields)

winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4)


27137

In [12]:
len(pc_fields)

27137

## 3, Alpha factory
### start with First Order

In [13]:
first_order = get_first_order(pc_fields, ts_ops)
print(first_order[:10])
print(len(first_order))

['winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4)', 'ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 'ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 22)', 'ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 66)', 'ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 120)', 'ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 240)', 'ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 'ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 22)', 'ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 66)', 'ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 120)']
2333782


In [14]:
# Pad initial decay with alpha
init_decay = 4
fo_alpha_list = []
for alpha in first_order:
    fo_alpha_list.append((alpha, init_decay))
print(len(fo_alpha_list))
print(fo_alpha_list[:5])

2333782
[('winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 22)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 66)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 120)', 4)]


In [10]:
# Load alphas to task pools
pools = load_task_pool(fo_alpha_list, 10, 9)
print(pools[0])

[[('winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 22)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 66)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 120)', 4), ('ts_rank(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 240)', 4), ('ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 4), ('ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 22)', 4), ('ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 66)', 4), ('ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 120)', 4)], [('ts_zscore(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 240)', 4), ('ts_delta(winsorize(ts_backfill(anl14_actvalue_bvps_fp0, 120), std=4), 5)', 4), ('ts_delta(winsorize(ts_b

In [ ]:
len(pools)

## 4, simulate alphas

In [ ]:
# Simulate First Order
multi_simulate(pools, "SUBINDUSTRY", "USA", "TOP3000", 0)

b'{"user":{"id":"TJ91365"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
pool 0 task 8 post done


## 5, Select alphas
go to web alphas penal to look for the number and date to track for next order improve

In [13]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("12-12", "12-12", 1.2, 0.5, "USA", 1000, "track")

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
100
200
300
400
500
600
700
800
900
count: 0


## 6, Next order improvement - Second Order
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [15]:
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))

fo_layer = prune(fo_tracker['next'] + fo_tracker['decay'], 'USA', 'mdl26', 5)
so_alpha_list = []
group_ops = group_ops = ["group_neutralize", "group_rank", "group_normalize", "group_scale", "group_zscore"]
for region, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], group_ops, region):
            so_alpha_list.append((alpha,decay))

print(len(so_alpha_list))
print(so_alpha_list[:3])

0
0
0
[]


### Simulate second order

In [16]:
so_pools = load_task_pool(so_alpha_list, 9, 10)
multi_simulate(so_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
Simulate done


## Higher Order for improvement - Third Order
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [17]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas("12-12", "12-12", 1.4, 0.7, "USA", 360, "track")

print(len(so_tracker['next']))
print(len(so_tracker['decay']))

so_layer = prune(so_tracker['next'] + so_tracker['decay'], 'chn', 'mdl26', 5)
th_alpha_list = []
for region, couples in so_layer.items():
    for expr, decay in couples:
        for alpha in trade_when_factory("trade_when",expr,region):
            th_alpha_list.append((alpha,decay))
print(len(th_alpha_list))

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
100
200
300
count: 0
0
0
0


### Simulate Third Order

In [19]:
# Simulate third order
th_pools = load_task_pool(th_alpha_list, 9, 10)
multi_simulate(th_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
Simulate done


## 7, Get submittable alphas
check_submission and view alphas

In [21]:
# get submitable alphas to check submission
th_tracker = get_alphas("12-12", "12-12", 1.58, 1, "USA", 200, "submit")

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
0
100
count: 0


In [22]:
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

0
b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'
[]


[]

In [23]:
# look date and metrics to locate alphas in the web
view_alphas(gold_bag)

b'{"user":{"id":"DZ43600"},"token":{"expiry":14400.0},"permissions":["CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","VISUALIZATION","WORKDAY"]}'


## 8, fine-tune submittable alphas
neutralization, performance comparison, turnover, margin